# S1 / S2 鉴别插件演示

本 notebook 展示三部分内容：

1) **WaveformWidthPlugin** 计算宽度
2) **GroupedEventsPlugin** 进行事件分组
3) **S1S2ClassifierPlugin** 进行 S1 / S2 鉴别

下面使用**简化的合成数据**演示插件调用方式。真实数据只需替换数据获取部分。


In [1]:
import numpy as np
import pandas as pd

from waveform_analysis.core.context import Context
from waveform_analysis.core.processing.dtypes import create_record_dtype
from waveform_analysis.core.plugins.builtin.cpu import (
    BasicFeaturesPlugin,
    GroupedEventsPlugin,
    S1S2ClassifierPlugin,
    WaveformWidthPlugin,
)
from waveform_analysis.core.plugins.builtin.cpu.peak_finding import (
    ADVANCED_PEAK_DTYPE,
)


## 1) 计算波形宽度 + 基础特征

这里构造一个简化的 `st_waveforms` 和手工 `signal_peaks`，
用于演示 **WaveformWidthPlugin** 和 **BasicFeaturesPlugin**。


In [2]:
from waveform_analysis import DAQAnalyzer


analyzer = DAQAnalyzer("/mnt/data/Run3/DAQ")  # Initialize DAQAnalyzer for proper waveform handling


analyzer.scan_all_runs()
analyzer.display_overview()

运行名称,文件数,大小(MB),大小,通道数,通道列表,采集开始,采集结束,路径
45V_OV_circulation_CH0_Coincidence_20dB,7,4448.09,4.34 GB,3,"0, 2, 3",2025-12-25 20:36:27,2025-12-25 20:45:32,/mnt/data/Run3/DAQ/45V_OV_circulation_CH0_Coincidence_20dB
46V_OV_circulation_CH0_Coincidence_20dB,7,4463.57,4.36 GB,3,"0, 2, 3",2025-12-25 20:16:17,2025-12-25 20:25:24,/mnt/data/Run3/DAQ/46V_OV_circulation_CH0_Coincidence_20dB
47V_OV_circulation_CH0_Coincidence_20dB,7,4471.24,4.37 GB,3,"0, 2, 3",2025-12-25 19:58:36,2025-12-25 20:07:45,/mnt/data/Run3/DAQ/47V_OV_circulation_CH0_Coincidence_20dB
48V_OV_circulation_CH0_Coincidence_20dB,20,19253.28,18.80 GB,3,"0, 2, 3",2025-12-25 18:45:32,2025-12-25 19:49:43,/mnt/data/Run3/DAQ/48V_OV_circulation_CH0_Coincidence_20dB
49V_OV_circulation_20thr,7,5742.33,5.61 GB,2,"6, 7",2025-12-24 20:10:41,2025-12-25 14:37:41,/mnt/data/Run3/DAQ/49V_OV_circulation_20thr
49V_OV_circulation_CH0_Coincidence,0,0.00,0.00 B,0,-,N/A,N/A,/mnt/data/Run3/DAQ/49V_OV_circulation_CH0_Coincidence
49V_OV_circulation_CH0_Coincidence_20dB,15,12136.51,11.85 GB,3,"0, 2, 3",2025-12-25 17:58:51,2025-12-25 18:36:30,/mnt/data/Run3/DAQ/49V_OV_circulation_CH0_Coincidence_20dB
49V_OV_circulation_CH0_Coincidence_27dB,4,2117.15,2.07 GB,3,"0, 2, 3",2025-12-25 17:45:55,2025-12-25 17:46:26,/mnt/data/Run3/DAQ/49V_OV_circulation_CH0_Coincidence_27dB
50V_OV_circulation_20thr,6,5655.19,5.52 GB,2,"6, 7",2025-12-23 06:16:10,2025-12-24 11:29:23,/mnt/data/Run3/DAQ/50V_OV_circulation_20thr
50V_over_voltage,17,16127.07,15.75 GB,2,"6, 7",2025-12-18 22:16:22,2025-12-19 00:08:04,/mnt/data/Run3/DAQ/50V_over_voltage


In [ ]:
run_id = "demo_run"
ctx = Context(storage_dir="./_demo_cache")

ctx.set_config({
    "data_root": "/mnt/data/Run3/DAQ",
    "daq_adapter": "vx2730",  # 文件名包含 @VX2730，需要适配器
    # 配置 raw_files 插件
    "raw_files.n_channels": 2,  # 实际只有 2 个通道有数据
    "raw_files.start_channel_slice": 2,  # 从通道 2 开始（修复：改为 2）
    # 配置 waveforms 插件
    "waveforms.n_channels": 2,  # 实际只有 2 个通道有数据（修复：改为 2）
    "waveforms.start_channel_slice": 2,  # 从通道 2 开始（修复：改为 2）
})


ctx.register(BasicFeaturesPlugin())
ctx.register(WaveformWidthPlugin())
ctx.register(S1S2ClassifierPlugin())


wave_length = 200
dtype = create_record_dtype(wave_length)
st_ch = np.zeros(2, dtype=dtype)

for idx, peak_pos in enumerate([60, 120]):
    wave = np.ones(wave_length, dtype=np.float32) * 10.0
    wave += 90.0 * np.exp(-((np.arange(wave_length) - peak_pos) ** 2) / (2 * 10**2))
    st_ch[idx]["wave"] = wave
    st_ch[idx]["baseline"] = 10.0
    st_ch[idx]["timestamp"] = 1_000_000 + idx * 200_000  # ps
    st_ch[idx]["event_length"] = wave_length
    st_ch[idx]["channel"] = 0

ctx._set_data(run_id, "st_waveforms", [st_ch])

peaks = np.zeros(2, dtype=ADVANCED_PEAK_DTYPE)
for idx, peak_pos in enumerate([60, 120]):
    peaks[idx]["position"] = peak_pos
    peaks[idx]["timestamp"] = st_ch[idx]["timestamp"]
    peaks[idx]["channel"] = 0
    peaks[idx]["event_index"] = idx

ctx._set_data(run_id, "signal_peaks", [peaks])

ctx.set_config({"sampling_rate": 1.0}, plugin_name="waveform_width")

widths = ctx.get_data(run_id, "waveform_width")
features = ctx.get_data(run_id, "basic_features")

widths[0]


## 2) S1 / S2 鉴别

使用 `WaveformWidthPlugin` 的宽度 + `BasicFeaturesPlugin` 的面积/高度进行判断。


In [ ]:
ctx.set_config(
    {
        "width_unit": "ns",
        "s1_width_range": (0.0, 80.0),
        "s2_width_range": (200.0, None),
        "s1_area_range": (0.0, 120.0),
        "s2_area_range": (200.0, None),
        "conflict_policy": "unknown",
    },
    plugin_name="s1_s2",
)

labels = ctx.get_data(run_id, "s1_s2")
labels[0]


## 3) 事件分组 (GroupedEventsPlugin)

这里直接构造一个简化的 `df`，展示分组插件的使用方式。
`timestamp` 单位为 **ps**，分组窗口参数为 **ns**。


In [ ]:
ctx.register(GroupedEventsPlugin())

df = pd.DataFrame({
    "timestamp": [1_000_000, 1_050_000, 2_000_000, 2_060_000],
    "channel": [0, 1, 0, 1],
    "area": [50.0, 55.0, 500.0, 520.0],
    "height": [10.0, 11.0, 25.0, 27.0],
})

ctx._set_data(run_id, "df", df)
ctx.set_config({"time_window_ns": 100.0}, plugin_name="df_events")
ctx.set_config({"n_channels": 2, "start_channel_slice": 0})

df_events = ctx.get_data(run_id, "df_events")
df_events


## 真实数据流程提示

如果你有真实的 DAQ 数据，只需把 `st_waveforms` / `signal_peaks` / `df` 的构造，
替换为真实数据管线（RawFiles → Waveforms → StWaveforms → Features → DataFrame）。

例如（示意，不执行）：

```python
from waveform_analysis.core.plugins.builtin.cpu import (
    RawFilesPlugin, WaveformsPlugin, SignalPeaksPlugin,
    BasicFeaturesPlugin, WaveformWidthPlugin, S1S2ClassifierPlugin,
    DataFramePlugin, GroupedEventsPlugin,
)

ctx = Context(config={"data_root": "DAQ", "n_channels": 2})
ctx.register(RawFilesPlugin())
ctx.register(WaveformsPlugin())
ctx.register(SignalPeaksPlugin())
ctx.register(BasicFeaturesPlugin())
ctx.register(WaveformWidthPlugin())
ctx.register(S1S2ClassifierPlugin())
ctx.register(DataFramePlugin())
ctx.register(GroupedEventsPlugin())

labels = ctx.get_data("run_001", "s1_s2")
df_events = ctx.get_data("run_001", "df_events")
```
